IMPORTANT:this TF file Needs MP_Data folder from data collection step

In [ ]:
#To remove leftovers from previous training
#!rm -rf MP_Data_Child
#!rm -rf MP_Data_Child.zip
#!rm -rf action.h5
#!rm -rf Logs

# Preprocess

In [361]:
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [367]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
#import mediapipe as mp

In [368]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
#Upload the zipped data
#!unzip file_location
!unzip MP_Data_Child.zip

In [409]:
# Path for exported data, numpy arrays
Folder='MP_Data_Child'

DATA_PATH = os.path.join(Folder) 

#Sorting to negate shuffling of dirs
dirs=os.listdir(Folder)
dirs.sort()

# Actions that we try to detect
actions = np.array(dirs)

# Thirty videos worth of data
no_sequences = len(os.listdir(os.path.join(Folder,actions[0])))

# Videos are going to be 24 frames in length
sequence_length = len(os.listdir(os.path.join(Folder,actions[0],str(0))))


In [410]:
label_map = {label:num for num, label in enumerate(actions)}

In [411]:
label_map

{'Hello': 0, 'ILoveYou': 1, 'No': 2, 'ThankYou': 3, 'Yes': 4}

In [412]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [413]:
#was able to condense info from 1662 datapoints to 261

In [414]:
#np.array(labels)

In [415]:
X = np.array(sequences)
X.shape

(500, 24, 225)

In [416]:
y = to_categorical(labels).astype(int)
#y

In [417]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [418]:
X_train.shape, X_test.shape

((450, 24, 225), (50, 24, 225))

# Train

In [419]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

TensorBoard

In [420]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

LSTM Training Model

In [421]:
X.shape

(500, 24, 225)

In [422]:
model = Sequential()
#3 sets of LSTM layers
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(X.shape[1],X.shape[2])))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
#2 Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
#Softmax function
model.add(Dense(actions.shape[0], activation='softmax'))

In [423]:
#(X.shape[1],X.shape[2])

In [424]:
#actions.shape[0]

In [425]:
#sidework
#res = [.7, .3]
#actions[np.argmax(res)]

In [426]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [427]:
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])

Epoch 1/200
15/15 [==============================] - 4s 63ms/step - loss: 1.5306 - categorical_accuracy: 0.2844
Epoch 2/200
15/15 [==============================] - 1s 68ms/step - loss: 1.4059 - categorical_accuracy: 0.4444
Epoch 3/200
15/15 [==============================] - 1s 67ms/step - loss: 1.3275 - categorical_accuracy: 0.4111
Epoch 4/200
15/15 [==============================] - 1s 63ms/step - loss: 1.1469 - categorical_accuracy: 0.5422
Epoch 5/200
15/15 [==============================] - 1s 66ms/step - loss: 0.9900 - categorical_accuracy: 0.6111
Epoch 6/200
15/15 [==============================] - 1s 64ms/step - loss: 0.7411 - categorical_accuracy: 0.6822
Epoch 7/200
15/15 [==============================] - 1s 66ms/step - loss: 0.5101 - categorical_accuracy: 0.7800
Epoch 8/200
15/15 [==============================] - 1s 65ms/step - loss: 0.5143 - categorical_accuracy: 0.7511
Epoch 9/200
15/15 [==============================] - 1s 65ms/step - loss: 0.4085 - categorical_accuracy:

Epoch 200/200
15/15 [==============================] - 1s 68ms/step - loss: 0.0180 - categorical_accuracy: 0.9911

In [442]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_24 (LSTM)              (None, 24, 64)            74240     
                                                                 
 lstm_25 (LSTM)              (None, 24, 128)           98816     
                                                                 
 lstm_26 (LSTM)              (None, 64)                49408     
                                                                 
 dense_24 (Dense)            (None, 64)                4160      
                                                                 
 dense_25 (Dense)            (None, 32)                2080      
                                                                 
 dense_26 (Dense)            (None, 5)                 165       
                                                                 
Total params: 228,869
Trainable params: 228,869
Non-tr

In [443]:
#tensorboard log dir
#!zip -r /content/file.zip /content/Logs
#from google.colab import files
#files.download("/content/file.zip")
#go to the correct command directory for Logs and type in the following in the command line..
#cd ..filedir
#tensorboard --logdir=.
#..then copy pase the URL to a browser

  adding: content/Logs/ (stored 0%)
  adding: content/Logs/train/ (stored 0%)
  adding: content/Logs/train/events.out.tfevents.1664997602.8abb4265d2df.55.9.v2 (deflated 92%)
  adding: content/Logs/train/events.out.tfevents.1664997858.8abb4265d2df.55.10.v2 (deflated 92%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Testing

In [444]:
result=model.predict(X_test)

In [445]:
actions[np.argmax(result[0])]

'Yes'

In [446]:
#Self evaluation
#i=0
#(actions[np.argmax(result[i])]==
#actions[np.argmax(y_test[i])])

In [447]:
model.save('action.h5')

In [448]:
#to save to local device
#from google.colab import files
#files.download("/content/action.h5")

In [449]:
#To delete model
#del model

In [450]:
model.load_weights('action.h5')

# Evaluation

In [451]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [452]:
yhat = model.predict(X_test)

In [453]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [454]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[41,  1],
        [ 0,  8]],

       [[39,  0],
        [ 1, 10]],

       [[39,  0],
        [ 0, 11]],

       [[40,  1],
        [ 1,  8]],

       [[39,  0],
        [ 0, 11]]])

In [455]:
accuracy_score(ytrue, yhat)

0.96